In [2]:
from playwright.sync_api import sync_playwright
import imapclient
import pyzmail

In [3]:
def fetch_otp_from_email():
    with imapclient.IMAPClient('imap.gmail.com') as client:
        client.login('your_email@example.com', 'your_password')
        client.select_folder('INBOX')
        messages = client.search(['UNSEEN'])
        for msgid, data in client.fetch(messages, 'RFC822').items():
            email_message = pyzmail.PyzMessage.factory(data[b'RFC822'])
            if 'Your OTP' in email_message.get_subject():
                otp_code = extract_otp_from_email(email_message.text_part.get_payload().decode('utf-8'))
                return otp_code

In [4]:
def extract_otp_from_email(email_content):
    # This is a placeholder. Use regex or simple string methods to extract the OTP.
    otp_code = email_content.strip()  # Simplistic, refine as needed
    return otp_code

In [9]:
def run(playwright):
    browser = playwright.chromium.launch(headless=True)
    page = browser.new_page()
    page.goto('https://prenotami.esteri.it/UserArea')

    # Login process
    page.fill('#email', 'your_email@example.com')
    page.click('#request_otp')

    # Fetch OTP from email
    otp_code = fetch_otp_from_email()
    page.fill('#otp', otp_code)
    page.click('#submit')

    # Navigate to reservation section
    page.click('a[href="/ReservationLink"]')
    page.click('a:has-text("Third Option")')

    # Check for availability
    if page.is_visible('.no-availability-popup'):
        print("No availability found.")
    else:
        print("Availability found, selecting date.")
        available_date = page.query_selector('.calendar-green')
        available_date.click()

    browser.close()

In [5]:

with sync_playwright() as playwright:
    run(playwright)


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.